# Creating and Evaluating Predictors: Part 2 - Related Time Series

This notebook will build off of all the ealrier work and requires that at least the importing of target time series and related time series data be complete. If you have not performed those steps yet, go back, do so, then continue.

At this point you now have a target-time-series dataset and a related-time-series dataset loaded into a singular Dataset Group, this is what is required to leverage the models that support related data in Amazon Forecast. If your data supports item level metadata it could be added to the dataset group as well and would benefit only algorithms that support that (e.g. CNN-QR, DeepAR+, but **not** Prophet). 

To continue the work, start with the imports, determine your region, establish your API connections, and load all previously stored values

In [ ]:
# Python Built-Ins:
import json

# External Dependencies:
import boto3
from IPython.display import Markdown
import pprint

# Local Dependencies:
import util

In [ ]:
with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
    data = json.load(notebook_info)
    resource_arn = data['ResourceArn']
    region = resource_arn.split(':')[3]
print(region)

In [ ]:
session = boto3.Session(region_name=region)
forecast = session.client(service_name='forecast')
forecast_query = session.client(service_name='forecastquery')

In [ ]:
%store -r

## Creating and Training Predictors
 
Given that that our data is hourly and we want to generate a forecast on the hour, Forecast limits us to a horizon of 500 of whatever the slice is. This means we will be able to predict about 20 days into the future.

The cells below will define a few variables to be used with all of our models. Then there will be an API call to create each `Predictor` where they are based on Prophet and DeepAR+ respectfully.


In [ ]:
forecast_horizon = 240
num_backtest_windows = 1
backtest_window_offset = 240
forecast_frequency = "H"

In [ ]:
prophet_algorithm_arn = "arn:aws:forecast:::algorithm/Prophet"
cnnqr_algorithm_arn = "arn:aws:forecast:::algorithm/CNN-QR"

### Prophet

In [ ]:
# Prophet Specifics
# Note the REL to indicate related time series data
prophet_predictor_name = project + "_prophet_rel_algo_1"

In [ ]:
# Build Prophet:
prophet_create_predictor_response = forecast.create_predictor(
    PredictorName=prophet_predictor_name,
    AlgorithmArn=prophet_algorithm_arn,
    ForecastHorizon=forecast_horizon,
    PerformAutoML=False,
    PerformHPO=False,
    EvaluationParameters={
        "NumberOfBacktestWindows": num_backtest_windows,
        "BackTestWindowOffset": backtest_window_offset,
    },
    InputDataConfig={
        "DatasetGroupArn": datasetGroupArn,
        "SupplementaryFeatures": [
            { "Name": "holiday", "Value": "US" }
        ],
    },
    FeaturizationConfig={
        "ForecastFrequency": forecast_frequency,
        "Featurizations": [
            {
                "AttributeName": "target_value",
                "FeaturizationPipeline": [
                    {
                        "FeaturizationMethodName": "filling",
                        "FeaturizationMethodParameters": {
                            "frontfill": "none",
                            "middlefill": "zero",
                            "backfill": "zero",
                        },
                    },
                ],
            },
        ],
    },
)


### CNN-QR

In [ ]:
# CNN-QR Specifics
cnnqr_predictor_name = project + "_cnnqr_rel_algo_1"

In [ ]:
# Build CNN-QR:
cnnqr_create_predictor_response = forecast.create_predictor(
    PredictorName=cnnqr_predictor_name,
    AlgorithmArn=cnnqr_algorithm_arn,
    ForecastHorizon=forecast_horizon,
    PerformAutoML=False,
    PerformHPO=False,
    EvaluationParameters={
        "NumberOfBacktestWindows": num_backtest_windows,
        "BackTestWindowOffset": backtest_window_offset,
    },
    InputDataConfig={
        "DatasetGroupArn": datasetGroupArn,
        "SupplementaryFeatures": [
            { "Name": "holiday", "Value": "US" }
        ],
    },
    FeaturizationConfig={
        "ForecastFrequency": forecast_frequency,
        "Featurizations": [
            {
                "AttributeName": "target_value",
                "FeaturizationPipeline": [
                    {
                        "FeaturizationMethodName": "filling",
                        "FeaturizationMethodParameters": {
                            "frontfill": "none",
                            "middlefill": "zero",
                            "backfill": "zero",
                        },
                    },
                ],
            },
        ],
    },
)


Normally in our notebooks we would have a while loop that polls for each of these to determine the status of the models in training. For simplicity sake here we are going to rely on you opening a new browser tab and following along in the console until a predictor has been created for each algorithm. 

Your previous tab from opening this session of Jupyter Lab should still be open, from there navigate to the Amazon Forecast service page, then select your dataset group. Lastly click `Predictors` and you should see the creation in progress. Once they are active you are ready to continue.

## Examining the Predictors

Once each of the Predictors is in an `Active` state you can get metrics about it to better understand its accuracy and behavior. These are computed based on the hold out periods we defined when building the Predictor. The metrics are meant to guide our decisions when we use a particular Predictor to generate a forecast

### Prophet

Here we are going to look to see the metrics from this Predictor like the earlier sessions, we will now add the related data metrics to the table from the previous notebook as well.

In [ ]:
# Prophet Metrics
prophet_arn = prophet_create_predictor_response['PredictorArn']
prophet_rd_metrics = forecast.get_accuracy_metrics(PredictorArn=prophet_arn)
pp = pprint.PrettyPrinter()
pp.pprint(prophet_rd_metrics)
prophet_rd_RMSEs= util.extract_json_values(prophet_rd_metrics, 'RMSE')
markdown_results.append(prophet_rd_RMSEs[0])
prophet_rd_loss_values = util.extract_json_values(prophet_rd_metrics, "LossValue")
markdown_results = markdown_results + prophet_rd_loss_values[::-1][:3]

In [ ]:
Markdown("""
Here we see an RMSE of {0[12]} which is better than the original 
RMSE indicating that we may be best served using related data for this algorithm.

| Predictor | RMSE               | 10%                 | 50%                 | 90%                |
|-----------|--------------------|---------------------|---------------------|--------------------|
| ARIMA     | {0[0]}             | {0[1]}              | {0[2]}              | {0[3]}             |
| Prophet   | {0[4]}             | {0[5]}              | {0[6]}              | {0[7]}             |
| Prophet + Related Data| {0[12]}| {0[13]}             | {0[14]}             | {0[15]}            |
| CNN-QR    | {0[8]}             | {0[9]}              | {0[10]}             | {0[11]}            |

When digging into the metrics the quantile improvements are mostly marginal. Next let us see how CNN-QR performed.
""".format(markdown_results))

### CNN-QR

Same as Prophet, now you should look at the metrics from it.

In [ ]:
# CNN-QR Metrics
cnnqr_arn = cnnqr_create_predictor_response["PredictorArn"]
cnnqr_rd_metrics = forecast.get_accuracy_metrics(PredictorArn=cnnqr_arn)
pp = pprint.PrettyPrinter()
pp.pprint(cnnqr_rd_metrics)
cnnqr_rd_RMSEs = util.extract_json_values(cnnqr_rd_metrics, "RMSE")
markdown_results.append(cnnqr_rd_RMSEs[0])
cnnqr_rd_loss_values = util.extract_json_values(cnnqr_rd_metrics, "LossValue")
markdown_results = markdown_results + cnnqr_rd_loss_values[::-1][:3]

In [ ]:
Markdown("""
Now after training with CNN-QR we can seen an RMSE of {0[16]} which is still not ideal but the full break down is:

| Predictor | RMSE               | 10%                 | 50%                 | 90%                |
|-----------|--------------------|---------------------|---------------------|--------------------|
| ARIMA     | {0[0]}             | {0[1]}              | {0[2]}              | {0[3]}             |
| Prophet   | {0[4]}             | {0[5]}              | {0[6]}              | {0[7]}             |
| Prophet + Related Data| {0[12]}| {0[13]}             | {0[14]}             | {0[15]}            |
| CNN-QR    | {0[8]}             | {0[9]}              | {0[10]}             | {0[11]}            |
| CNN-QR + Related Data | {0[16]}| {0[17]}             | {0[18]}             | {0[19]}            |

In our test, adding related data actually degraded the performance of CNN-QR. Prophet with related data achieved the
lowest combined losses on p10 and p90 quantiles, but for predicting the median and minimising RMSE, CNN-QR without
related data was most accurate.

Additional work would need to be kicked off from here to determine the specific impact of these figures and how they
compare to the existing Forecasting approaches performed by your customer.
""".format(markdown_results))